In [ ]:
import zipfile
with zipfile.ZipFile('data_final.zip') as zf:
    zf.extractall('')

In [ ]:
!mkdir -p ./tools
%cd ./tools

!wget https://github.com/downloads/chokkan/liblbfgs/liblbfgs-1.10.tar.gz
!tar -xzf liblbfgs-1.10.tar.gz
%cd liblbfgs-1.10
!./configure --prefix=`pwd`
!make -j 6
!make -i install
%cd ../

!git clone https://github.com/BitSpeech/SRILM.git
%cd SRILM
!mv Makefile _Makefile
!echo "SRILM=$(pwd)" > Makefile
!cat _Makefile >> Makefile

!echo HAVE_LIBLBFGS=1 >> common/Makefile.machine.i686-m64
!grep ADDITIONAL_INCLUDES common/Makefile.machine.i686-m64 | \
    sed 's|$| -I$(SRILM)/../liblbfgs-1.10/include|' \
    >> common/Makefile.machine.i686-m64

!grep ADDITIONAL_LDFLAGS common/Makefile.machine.i686-m64 | \
    sed 's|$| -L$(SRILM)/../liblbfgs-1.10/lib/ -Wl,-rpath -Wl,$(SRILM)/../liblbfgs-1.10/lib/|' \
    >> common/Makefile.machine.i686-m64

!make -j 6
%cd ../../

In [ ]:
!pip install -q opencv-python==4.5.4.60
!pip install -qU albumentations
!pip install -q git+https://github.com/parlance/ctcdecode.git

     |████████████████████████████████| 60.3 MB 56 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 102 kB 9.0 MB/s 
     |████████████████████████████████| 47.7 MB 1.9 MB/s 


In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from google.colab.patches import cv2_imshow
import albumentations as albu

import numpy as np
import pandas as pd
import cv2
import os
import json
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import random
import editdistance
from torch import autocast
from ctcdecode import CTCBeamDecoder

In [ ]:
df = pd.read_csv('/content/data_final/train_recognition/labels.csv')
file_names, texts = df.file_name.values.tolist(), df.text.values.tolist()

In [ ]:
rus_texts, eng_texts = [], []
for i in range(len(df)):
    text = texts[i]
    if text not in ['2_?', '***.'] and "<" not in text:
        text = texts[i]
        text = text.replace('Ы', 'ы')
        text = text.replace('Ь', 'ь')
        text = text.replace('}', ']')
        text = text.replace('_', ')')
        text = text.replace('X', 'x')
        text = text.replace('K', 'k')
        text = text.replace('<<', '"')
        text = text.replace('>>', '"')
        text = text.replace('A', 'А')
        text = text.replace('a', 'а')
        text = text.replace('B', 'В')
        text = text.replace('C', 'С')
        text = text.replace('c', 'с')
        text = text.replace('E', 'Е')
        text = text.replace('e', 'е')
        text = text.replace('H', 'Н')
        text = text.replace('M', 'М')
        text = text.replace('m', 'м')
        text = text.replace('O', 'О')
        text = text.replace('o', 'о')
        text = text.replace('P', 'Р')
        text = text.replace('p', 'р')
        text = text.replace('T', 'Т')
        text = text.replace('K', 'К')
        text = text.replace('k', 'к')
        text = text.replace('X', 'Х')
        text = text.replace('x', 'х')
        if 'eng' in file_names[i]:
            eng_texts.append(text)
        else:
            rus_texts.append(text)

Взяли датасет 5к сочинений с ProЧтение https://ai.upgreat.one/participants/datasets/

https://drive.google.com/file/d/1hfCTx0FBh8X9xYcGDAMAsrbd-ZUnMyvO/view?usp=sharing

In [ ]:
!mkdir rus_essay
import zipfile
with zipfile.ZipFile('rus_essay.zip', 'r') as zf:
    zf.extractall('rus_essay')

In [ ]:
pro_soc = []
for path in tqdm(os.listdir('/content/rus_essay/ds-rus-train 2')):
    with open(f'/content/rus_essay/ds-rus-train 2/{path}', 'r') as f:
        data = json.load(f)
        pro_soc.append(data['text'].split(' '))
pro_soc = np.concatenate(pro_soc)

  0%|          | 0/29202 [00:00<?, ?it/s]

In [ ]:
def prepare_text(text):
    if text == '': return False

    if text[0] in 'I1234567890':
        return [text]

    text1, text2 = text, text
    text1 = text1[0].upper() + text1[1:]
    text2 = text2[0].lower() + text2[1:]

    if len(text) > 6 and ' ' not in text and '.' not in text[:-1] and ',' not in text[:-1] and (text[-1] == '.' or text[-1] == ','):
        return [text1[:-1]+',', text1[:-1]+'.', text1[:-1], text2[:-1]+',', text2[:-1]+'.', text2[:-1]]

    bad = [x for x in text if x in ' !|"\'()+,-./0123456789:;=?IN[]№']
    if not len(bad) and len(text) >= 5:
        return [text1, text2, text1+'.', text1+',', text2+'.', text1+',']

    return [text1, text2]

def get_additional_texts():
    texts = []
    texts.append(list(str(x) for x in range(1900, 2023)))
    texts.append(list(str(x) for x in range(1000)))
    texts.append(list('№'+str(x) for x in range(1000)))
    texts.append(list('упр'+str(x) for x in range(1000)))
    texts.append(np.concatenate(list(x.split(' ') for x in TEXT.split('\n') if len(x))))
    return np.concatenate(texts)

soc_texts = TEXT.split('\n')
soc_texts = np.concatenate(list(x.split(' ') for x in soc_texts))

train_texts = np.concatenate((rus_texts, pro_soc, get_additional_texts()))
train_texts = sorted(set(train_texts))
train_texts = sorted(set(np.concatenate(list(prepare_text(x) for x in train_texts if prepare_text(x)))))

!mkdir data

with open('/content/data/rus_texts.txt', 'w', encoding='utf-8') as f:
    print('\n'.join(train_texts), file=f)

!/content/tools/SRILM/bin/i686-m64/ngram-count -text /content/data/rus_texts.txt -order 6 -wbdiscount -interpolate -maxent -maxent-convert-to-arpa -lm /content/data/rus_language_model.gz

Starting fitting...
Starting OWL-BFGS with c1=2.0936e-07, sigma2=1.43294e+07, max_iters=1000
  No of NaNs in logZs: 0, No infs: 0
  dual is 13.8301
  regularized dual is 13.8301
  norm of gradient =0.499554
  norm of regularized gradient =0.499554
  No of NaNs in logZs: 0, No infs: 0
  dual is 13.3306
  regularized dual is 13.3306
  norm of gradient =0.499553
  norm of regularized gradient =0.499553
Iteration 1
  No of NaNs in logZs: 0, No infs: 1017112
  dual is inf
  regularized dual is inf
  norm of gradient =0.00311338
  norm of regularized gradient =0.019997
  No of NaNs in logZs: 0, No infs: 1017112
  dual is inf
  regularized dual is inf
  norm of gradient =0.0037434
  norm of regularized gradient =0.0105414
  No of NaNs in logZs: 0, No infs: 1017112
  dual is inf
  regularized dual is inf
  norm of gradient =0.00520336
  norm of regularized gradient =0.00713342
  No of NaNs in logZs: 0, No infs: 1017112
  dual is inf
  regularized dual is inf
  norm of gradient =0.00616769
  no

In [ ]:
english_texts = set(eng_texts)

Датасет английских сочинений https://www.kaggle.com/c/asap-aes

https://drive.google.com/file/d/17DJ8LjGciSZRxFfptZW-40TkAiFyKskx/view?usp=sharing

In [ ]:
with open('english_words_hewlett.txt', 'r') as fe:
    words = [x[:-2] for x in fe.readlines()]
    for word in words:
        english_texts.add(word)

In [ ]:
new_english_texts = []
eng_texts_old = sorted(english_texts)
for i in range(len(english_texts)):
    text = eng_texts_old[i]
    text = text.replace('Ы', 'ы')
    text = text.replace('Ь', 'ь')
    text = text.replace('}', ']')
    text = text.replace('_', ')')
    text = text.replace('X', 'x')
    text = text.replace('K', 'k')
    text = text.replace('<<', '"')
    text = text.replace('>>', '"')
    text = text.replace('A', 'А')
    text = text.replace('a', 'а')
    text = text.replace('B', 'В')
    text = text.replace('C', 'С')
    text = text.replace('c', 'с')
    text = text.replace('E', 'Е')
    text = text.replace('e', 'е')
    text = text.replace('H', 'Н')
    text = text.replace('M', 'М')
    text = text.replace('m', 'м')
    text = text.replace('O', 'О')
    text = text.replace('o', 'о')
    text = text.replace('P', 'Р')
    text = text.replace('p', 'р')
    text = text.replace('T', 'Т')
    text = text.replace('K', 'К')
    text = text.replace('k', 'к')
    text = text.replace('X', 'Х')
    text = text.replace('x', 'х')
    new_english_texts.append(text)

In [ ]:
with open('/content/data/eng_texts.txt', 'w', encoding='utf-8') as f:
    print('\n'.join(new_english_texts), file=f)

!/content/tools/SRILM/bin/i686-m64/ngram-count -text /content/data/eng_texts.txt -order 6 -wbdiscount -interpolate -maxent -maxent-convert-to-arpa -lm /content/data/eng_language_model.gz

Starting fitting...
Starting OWL-BFGS with c1=7.93701e-06, sigma2=377976, max_iters=1000
  No of NaNs in logZs: 0, No infs: 0
  dual is 10.357
  regularized dual is 10.357
  norm of gradient =0.499817
  norm of regularized gradient =0.499817
  No of NaNs in logZs: 0, No infs: 0
  dual is 9.85719
  regularized dual is 9.85719
  norm of gradient =0.499763
  norm of regularized gradient =0.49976
Iteration 1
  No of NaNs in logZs: 0, No infs: 31496
  dual is inf
  regularized dual is inf
  norm of gradient =0.00398903
  norm of regularized gradient =0.0234457
  No of NaNs in logZs: 0, No infs: 31496
  dual is inf
  regularized dual is inf
  norm of gradient =0.00398903
  norm of regularized gradient =0.0122227
  No of NaNs in logZs: 0, No infs: 31496
  dual is inf
  regularized dual is inf
  norm of gradient =0.00398903
  norm of regularized gradient =0.00702141
  No of NaNs in logZs: 0, No infs: 31496
  dual is inf
  regularized dual is inf
  norm of gradient =0.00398903
  norm of regular